In [1]:
from keras.models import load_model,Model
from keras.layers import *
import keras
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Reshape, BatchNormalization
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [6]:
def Conv2DBNDrRelu(filterSize,kernelSize,strideSize,dropRate,inputTensor):
    x = Conv2D(filters=filterSize,kernel_size=kernelSize,strides=strideSize)(inputTensor)
    x = BatchNormalization()(x)
    x = Dropout(dropRate)(x)
'''
Inp = Input(shape=(784,),dtype='float32')
x = BatchNormalization()(Inp)
x = Reshape((28,28,1), input_shape=(784,))(x)
x = UpSampling2D((5,5))(x)
x = Conv2D(3,(1,1),activation='relu',padding='same')(x)
'''
inceptV3 = InceptionV3(include_top=False,input_shape=(1,28,28))

ValueError: The input must have 3 channels; got `input_shape=(1, 28, 28)`

In [5]:
inceptV3.summary()
#x = inceptV3(x)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_97 (Conv2D)               (None, None, None, 32 864         input_3[0][0]                    
____________________________________________________________________________________________________
batch_normalization_97 (BatchNor (None, None, None, 32 96          conv2d_97[0][0]                  
____________________________________________________________________________________________________
activation_95 (Activation)       (None, None, None, 32 0           batch_normalization_97[0][0]     
___________________________________________________________________________________________

In [2]:
someLayer = inceptV3.get_layer('mixed8')

ValueError: You are trying to load a weight file containing 188 layers into a model with 190 layers.

In [ ]:
x = Flatten()(someLayer.output)
x = Dense(128, activation='relu', name = "Dense_3")(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu', name = "Dense_4")(x)
output = Dense(200, activation='softmax', name = "Outputlayer")(x)

model = Model(Inp, output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=2, min_lr=0.00001)
#learning_rate = 0.1
#K.set_value(model.optimizer.lr, learning_rate)

In [ ]:
inceptV3 = InceptionV3(include_top=False, input_shape=(165,65,1), weights='imagenet')

In [ ]:
inceptV3.summary()

In [ ]:

getInter = model.get_layer('mixed8').output

In [ ]:
x = Dense(4096,activation='relu',name='fcnew')(getInter)
x=Dropout(0.2,name='drnew')(x)
x = Dense(4096,activation='relu',name='fcnew1')(x)
x=Dropout(0.5,name='drnew1')(x)
x = Dense(200,activation='softmax',name = 'output')(x)

In [ ]:
from keras.models import Model
newModel = Model(model.input,x)
newModel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
num_examples_per_class = 5000


classes = ['aircraft carrier','airplane','alarm clock','ambulance','angel','ant',
'anvil','apple','axe','banana','bandage','barn','baseball bat',
'baseball','basket','basketball','bathtub','beach','bear','beard',
'bed','bee','belt','bicycle','binoculars','birthday cake',
'blueberry','book','boomerang','bottlecap','bowtie','bracelet',
'brain','bread','broom','bulldozer','bus','bush','butterfly',
'cactus','cake','calculator','calendar','camel','camera','campfire',
'candle','cannon','canoe','car','carrot','cat','cello','chandelier',
'clock','cloud','coffee cup','compass','computer','cookie','couch',
'cow','crab','crayon','crocodile','crown','cup','diamond','dog',
'dolphin','donut','dragon','dresser','drill','drums','duck','dumbbell','ear','elbow','elephant','envelope','eraser',
'eye','eyeglasses','face','fan','feather','fence','finger','fire hydrant',
'fireplace','firetruck','fish','flamingo','flashlight','flip flops',
'floor lamp','flower','flying saucer','foot','fork','frog','frying pan',
'garden hose','garden','giraffe','goatee','golf club','grapes','grass',
'guitar','hamburger','hammer','hand','harp','hat','headphones',
'hedgehog','helicopter','helmet','hexagon','hockey puck',
'hockey stick','horse','hospital','hot air balloon','hot dog',
'hot tub','hourglass','house plant','house','hurricane',
'ice cream','jacket','jail','kangaroo','key','keyboard','knee',
'knife','ladder','lantern','laptop','leaf','leg','light bulb',
'lighter','lighthouse','lightning','line','lion','lipstick','lobster',
'lollipop','mailbox','map','marker','matches','megaphone',
'mermaid','microphone','microwave','monkey',
'moon','mosquito','motorbike','mountain','mouse','moustache',
'mouth','mug','mushroom','nail','necklace','nose','ocean',
'octagon','octopus','onion','oven','owl',
'paint can','paintbrush','palm tree','panda','pants',
'paper clip','parachute','parrot','passport','peanut',
'pear','peas','pencil','penguin','piano','pickup truck',
'picture frame','pig','pillow']
x_data = np.load("./x_data_200_classes_5k.npy")

labels = [np.full((num_examples_per_class,), classes.index(qdraw)) for qdraw in classes]

In [ ]:
## Concat the arrays together
y_data = np.concatenate(labels,axis=0)
y_data.shape


y_data = keras.utils.to_categorical(y_data,200)

dataOrder =  np.random.permutation(x_data.shape[0])
trainIdx = dataOrder[0:np.floor(0.8*dataOrder.shape[0]).astype('int32')]
testIdx = dataOrder[(np.floor(0.8*dataOrder.shape[0]).astype('int32')+1):-1]

def myGeneratorBasic(x_data,y_data,batchSize,dataOrder):
    while True:
        np.random.shuffle(dataOrder)
        for i in range(0, len(dataOrder), batchSize):
            xTrain = x_data[dataOrder[i:i+batchSize]]
            yTrain = y_data[dataOrder[i:i+batchSize]]
            yield({'input_1':np.array(xTrain)},{'Outputlayer':np.array(yTrain)})

In [ ]:
history = model.fit_generator(myGeneratorBasic(x_data,y_data,2,trainIdx),400,50)